In [1]:
import os
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import sys
import math
from netCDF4 import Dataset
import os, calendar, sys, fnmatch, datetime
from sur_spec_alb_interpolate import sur_spec_alb_interpolate


from INPUT_RRTM_lw import *
from INPUT_CLD_RRTM_lw import *
from read_output_rrtm import *

In [6]:
#******************* Specify the path of compiled RRTM codes ****************
model_dir = '/umbc/xfs1/zzbatmos-new/common/Codes/'
output_dir = './rrtm_output/'
RT_model_lw = model_dir+'rrtm_lw/rrtm_lw_taki_intel'

#=======================================
#Specify parameters in the RRTM model
#=======================================

SOLVAR  = 0.0  #(16,29)the solar source function scale factor for each band.
#HBOUND = np.loadtxt(fname=file6)       #altitude of the surface (km)
HTOA   = 77.0       #altitude of the top of the atmosphere (km)

#--------------------------------------------------;
ISCAT_sw   = 0
ISCAT_lw   = 2
NUMANGS = 0 # 4 streams for lw
ISTRM = 0   # 4 streams for sw
NSTR  = 4   # of streams

In [7]:
# =====RRTM input data (atmospheric and surface properties)=====
# ***Note: RRTM requires Altitude (ZM) to be in a ascending order!!! (Make sure the vertical order of all other variables is consistent with the order of ZM)
# ***Note: Be careful about the unit of each parameter (Check JCHAR variable in RRTM_SW instruction)

atm_prof = np.loadtxt('atm_profile_aerosol.txt',skiprows=1)
ZM = atm_prof[::-1,0] #km (boundary altitude (km))
PM = atm_prof[::-1,1] #hpa
TM = atm_prof[::-1,2] #K
h2o = atm_prof[::-1,4]  #g/m3   #kg/kg . kg/kg *1e3 = gm/kg
o3 = atm_prof[::-1,5]   #g/m3   #ppmv 'parts per million volume = 1e-6'. 1ppmv=1e-3*gm/kg
dext = atm_prof[::-1,6] #(km-1)

Es = np.array([0.1]) # surface Emissivity
Ts = np.array([295]) # surface Temperature (unit: K)

In [8]:
# ===== Aerosol (e.g., dust) optical properties (i.e., Qext, SSA, g) =====
# ***Note: The order of 14 bands is listed in TABLE 1 in RRTM_SW instruction (It is not monotonically increase or decrease)
# ***Note: The order of spectral optical properties should be consistent with the order in TABLE 1

opt_dir = '../../Dust_Optical_Properties/'
opt_prop_sw = np.loadtxt(opt_dir + 'opt_prop_nonsphere_sal_sw.txt',skiprows=1)
qe_dust_sw = opt_prop_sw[:,0]  # Dust extinction efficienty (Qe) for 14 RRTM_SW bands
ssa_dust_sw = opt_prop_sw[:,1] # Dust single scattering albedo (SSA) for the 14 RRTM_SW bands
g_dust_sw = opt_prop_sw[:,2]   # Dust assymetric parameter (g) for the 14 RRTM_SW bands

opt_prop_lw = np.loadtxt(opt_dir + 'opt_prop_nonsphere_sal_lw.txt',skiprows=1)
qe_dust_lw = opt_prop_lw[:,0]  # Dust extinction efficienty (Qe) for 16 RRTM_LW bands
ssa_dust_lw = opt_prop_lw[:,1] # Dust single scattering albedo (SSA) for the 16 RRTM_LW bands
g_dust_lw = opt_prop_lw[:,2]   # Dust assymetric parameter (g) for the 16 RRTM_LW bands

pmom_dust = np.zeros((16,NSTR+1))
for i in range(NSTR+1):
    pmom_dust[:,i] = g_dust_lw**i

In [9]:
n_case = np.size(Es)
print('*****Number of cases:',n_case)

for ic in range(n_case):
    SEMISS = np.ones(14)*Es[ic]
    TBOUND = Ts
    if ZM[0]<0.1 :
        HBOUND = ZM[0]+0.001
    else:
        HBOUND = ZM[0]+0.01
    IMMAX = len(PM)
    IBMAX = IMMAX

    VMOL_wv = h2o   ###kg/kg*1e3=gm/kg
    VMOL_o3 = o3    ###ppmv*1e3=gm/kg
    VMOL_co2= np.ones(IMMAX)* 360 #no CO2 profile in data, add mannually, ppmv
    VMOL_n2o = np.ones(IMMAX)*0.3 #unit: PPMV
    VMOL_co = np.zeros(IMMAX)
    VMOL_ch4 = np.ones(IMMAX)*1.7 #unit: PPMV
    VMOL_total = np.vstack((VMOL_wv,VMOL_co2,VMOL_o3,VMOL_n2o,VMOL_co,VMOL_ch4))#density of the molecule set by JCHAR(K)
    VMOL=VMOL_total
    
    
    NLAY = np.count_nonzero(dext)   # Total number of layers containing the aerosol with the specified properties
    aer_lay = np.nonzero(dext)[0]+1 # Layer number  of aerosol layer. (layer n corresponds to the region between altitudes n and n+1 in the list of layer boundaries in Record 3.3B.)

    #----For each aerosol layer calculate AOD for each band----
    aod = np.zeros(NLAY)
    for i in range(NLAY):
        aod[i]=dext[aer_lay[i]-1]*(ZM[aer_lay[i]]-ZM[aer_lay[i]-1])
    aod_lay_band_lw=np.zeros((16,NLAY))
    for j in range(NLAY):
        aod_lay_band_lw[:,j]= aod[j] * qe_dust_lw/qe_dust_sw[9]   #qe_dust_sw[9] is for 550nm
    
    print('*****Altitude for RRTM_SW (unit: km):',ZM)
    
    #========================== Run RRTM_SW ====================================
    ICLD = 0 # Cloud-free
    print(' Compute clear-sky lw flux w/o cloud')
    read_input_rrtm_lw('INPUT_RRTM', ICLD, ISCAT_lw, NUMANGS, TBOUND, HBOUND, \
                       SEMISS, IMMAX, ZM, PM, TM, VMOL)
    os.system(RT_model_lw)
    os.system('mv INPUT_RRTM INPUT_RRTM.lw')
    os.system('mv OUTPUT_RRTM '+output_dir+'OUTPUT_RRTM_lw.clear')
    
    
    ICLD = 1   # Cloudy (Here we use Aerosol to represent Cloud)

    print(' Compute lw fluxs with cloud')
    read_input_rrtm_lw('INPUT_RRTM', ICLD, ISCAT_lw, NUMANGS, TBOUND, HBOUND, \
                       SEMISS, IMMAX, ZM, PM, TM, VMOL)

    write_IN_CLD_RRTM_lw('IN_CLD_RRTM', NLAY, aer_lay, aod_lay_band_lw, \
                                  ssa_dust_lw, NSTR, pmom_dust)
    os.system(RT_model_lw)
    os.system('mv INPUT_RRTM INPUT_RRTM.lw')
    os.system('mv IN_AER_RRTM IN_AER_RRTM.lw')
    os.system('mv OUTPUT_RRTM '+output_dir+'OUTPUT_RRTM_lw.cloud')

*****Number of cases: 1
*****Altitude for RRTM_SW (unit: km): [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 30. 35. 40. 45. 50.]
 Compute clear-sky lw flux w/o cloud
 Compute lw fluxs with cloud
